In [4]:
import pandas as pd
import functions as fn
import numpy as np
import matplotlib.pyplot as plt

In [5]:
tickers = pd.read_csv("../data/nasdaq_comps_2016-2020.csv")
tickers = list(set(tickers['2019']).intersection(set(tickers['2018'])))
etf = 'QQQ'
start = '2018-01-01'
end = '2018-12-31'

In [26]:
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

coint_data = pd.DataFrame(columns=['ticker', 'adf-stat', 'pval'])
etf_data = yf.download(etf, start = start, end = end)
etf_data = etf_data[['Close']]
etfLogPrice = np.log(etf_data['Close'].values)

for i, t in enumerate(tickers):
    t_data = yf.download(t, start = start, end = end)['Close']
    tLogPrice = np.log(t_data.values)
    if t_data.isnull().values.any() or len(etfLogPrice) != len(tLogPrice):
        print('err')
        continue
    X = sm.add_constant(tLogPrice)
    y = log_etf_data['Close'].values
    
    if len(X) != len(y):
        print("Error: length of datasets not equal")
        print(len(X), len(y))
        continue
    
    result = sm.OLS(y, X).fit()
    
    resid = result.resid
    adf_results = adfuller(resid)
    coint_data.loc[i] = [t, adf_results[0], adf_results[1]]

coint_data = coint_data.sort_values(by='adf-stat', ascending=True)
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

ticker  adf-stat      pval
56    HAS -3.469865  0.008796
81   AAPL -3.261963  0.016667
2    TTWO -3.180402  0.021148
72   IDXX -3.146207  0.023317
77   SBUX -3.119374  0.025152
..    ...       ...       ...
88   CSCO -0.904925  0.786346
51   ILMN -0.819534  0.813317
75   ISRG -0.506787  0.890685
16   MSFT -0.444993  0.902311
87   AMGN -0.430448  0.904890

[87 rows x 3 columns]

In [39]:
subset = coint_data.loc[coint_data['pval'] <= 0.05]
tickers = list(subset['ticker'])
subset

ticker  adf-stat      pval
56    HAS -3.469865  0.008796
81   AAPL -3.261963  0.016667
2    TTWO -3.180402  0.021148
72   IDXX -3.146207  0.023317
77   SBUX -3.119374  0.025152
25   CTAS -3.079973  0.028071
43   ALXN -2.892025  0.046283

In [45]:
import itertools
coint_data = pd.DataFrame(columns=['ticker', 'adf-stat', 'pval'])
tickers_subsets = []
tickers_data = yf.download(tickers, start = start, end = end)
for i in range(2, len(subset) + 1):
    for s in itertools.combinations(subset['ticker'].values, i):
        tickers_subsets += [s]

for i, t_list in enumerate(tickers_subsets):
        if i % 500 == 0:
            print(i, "done, out of a total of ", len(tickers_subsets))
        basket_sum = sum([tickers_data['Close'][t] for t in t_list])
        tLogPrice = np.log(basket_sum.values)
        if basket_sum.isnull().values.any() or len(etfLogPrice) != len(tLogPrice):
            print('err')
            continue
        X = sm.add_constant(tLogPrice)
        y = log_etf_data['Close'].values

        if len(X) != len(y):
            print("Error: length of datasets not equal")
            print(len(X), len(y))
            continue

        result = sm.OLS(y, X).fit()

        resid = result.resid
        adf_results = adfuller(resid)
        coint_data.loc[i] = [t_list, adf_results[0], adf_results[1]]

coint_data.sort_values(by='adf-stat', ascending=True).head(5)

[*********************100%***********************]  7 of 7 completed
0 done, out of a total of  120


ticker  adf-stat      pval
51       (TTWO, CTAS, ALXN) -4.882198  0.000038
71  (HAS, TTWO, CTAS, ALXN) -4.858668  0.000042
35        (HAS, CTAS, ALXN) -4.646081  0.000106
65  (HAS, AAPL, CTAS, ALXN) -4.613889  0.000122
28        (HAS, TTWO, CTAS) -4.550469  0.000159